## Analyze an Re sweep for Specific End States
### Orbit
### Monitor group velocity, distance, and angle between swimmers

In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Feb 14 14:16:04 2020

@author: thomas
"""

#MODULES
import os,sys
import re
import numpy as np
import pandas as pd
from mpl_toolkits import mplot3d
%matplotlib notebook
import matplotlib as mpl
#mpl.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator,AutoMinorLocator)
from scipy.signal import savgol_filter
import pathlib
from matplotlib import animation
from IPython.display import display, Image, HTML

mpl.rcParams['axes.linewidth'] = 1.5 #set the value globally

In [3]:
#CONSTANTS
cwd_PYTHON = os.getcwd()
PERIOD = 0.1
DT = 5.0e-3
RADIUSLARGE = 0.002
RADIUSSMALL = 0.001
FREQUENCY   = 10.0  

#Lists
#RLength
ReList=['0.5','1.0','2.0','3.0','4.0','5.0','5.5','6.0','6.5','7.0','7.5','10.0','12.5','15.0',
        '17.5','20.0','25.0','30.0','35.0','40.0','50.0','60.0']
allData = []

#### Obtain Pos Data for all simulations
#### Limit time to first five oscillations

In [4]:
def StoreData(cwd_POSDATA):
    #global axAll
    #Reset position data every Sim
    pdData = []
    #Load position data
    pdData = pd.read_csv(cwd_POSDATA+'/pd.txt',delimiter=' ')
    #Save only every 20 rows (Every period)
    pdData = pdData.iloc[::20]
    #Reindex so index corresponds to period number
    pdData = pdData.reset_index(drop=True)
    #Print pdData to make sure it has been done properly
    #print(pdData.head(6))
    #Create Swimmer A and Swimmer B dataframes
    dict_A = {'xU':pdData['aXU'],'yU':pdData['aYU'],'xL':pdData['aXL'],'yL':pdData['aYL'],'time':pdData['time']}
    dataA  = pd.DataFrame(data=dict_A)
    dict_B = {'xU':pdData['bXU'],'yU':pdData['bYU'],'xL':pdData['bXL'],'yL':pdData['bYL'],'time':pdData['time']}
    dataB  = pd.DataFrame(data=dict_B)
    #print(dataA)
    #print(dataB)
    #Create a swimmer class for each
    swimA = Swimmer(dataA)
    swimB = Swimmer(dataB)
    simTime = swimA.time
    angleA = swimA.get_theta()
    angleB = swimB.get_theta()
    #print('Theta_A = ',angleA*180.0/np.pi)
    #print('Theta_B = ',angleB*180.0/np.pi)
    
    #Calculate Combined CM
    xCM_comb, yCM_comb = CalcSystemCM(swimA.CM,swimB.CM)
    
    #Calculate the three parameters
    #Hx, Hy, and theta_BW in swimmer A's reference frame
    Hx, Hy, Hmag, theta_BW = swimB.CalcHxHyTheta(swimA)
    #print('Lab Frame')
    labHx, labHy = swimB.get_labH()
    #print('labHx = ',labHx)
    #print('labHy = ',labHy)
    #print('\n\n')
    #print('Hx = ',Hx)
    #print('Hy = ',Hy)
    #print('Theta_BW = ',180.0*theta_BW/np.pi)
    #First, create a dataframe to contain this information
    dict_Param = {'Hx':Hx,'Hy':Hy,'Hmag':Hmag,'Theta':theta_BW,
                  'xCM_comb':xCM_comb/RADIUSSMALL,'yCM_comb':yCM_comb/RADIUSSMALL,
                  'thetaA':angleA*180.0/np.pi,'thetaB':angleB*180.0/np.pi,
                  'xCMA':swimA.CM[0]/RADIUSSMALL,'yCMA':swimA.CM[1]/RADIUSSMALL,
                  'xCMB':swimB.CM[0]/RADIUSSMALL,'yCMB':swimB.CM[1]/RADIUSSMALL,
                  'time':simTime}
    parData = pd.DataFrame(data=dict_Param)
    parData['theta_deg'] = 180.0*parData['Theta']
    #print(parData)
    
    #Calculate Velocity of combined CM
    parData['vx_comb'], parData['vy_comb'], parData['v_comb'] = 0.0, 0.0, 0.0
    for idx in range(1,len(parData['time'])):
        parData.loc[idx,'vx_comb'] = (parData.loc[idx,'xCM_comb'] - parData.loc[idx-1,'xCM_comb'])/(PERIOD*FREQUENCY)
        parData.loc[idx,'vy_comb'] = (parData.loc[idx,'yCM_comb'] - parData.loc[idx-1,'yCM_comb'])/(PERIOD*FREQUENCY)
    parData['v_comb'] = np.hypot(parData['vx_comb'],parData['vy_comb'])
    
    #Calculate Angular Velocity for swimmers A and B
    #Angles are in range [0,360). Find using periodic conditions for 0 and 360
    parData['wA'], parData['wB'], parData['wAVG'] = 0.0, 0.0, 0.0
    for idx in range(1,len(parData['time'])):
        diffA1 = parData.loc[idx,'thetaA'] - parData.loc[idx-1,'thetaA']
        if(diffA1 <= -180.0):
            diffA = diffA1 + 360.0
        elif(diffA1 >= 180.0):
            diffA = diffA1 - 360.0
        else:
            diffA = diffA1
        parData.loc[idx,'wA'] = diffA/(PERIOD)
        diffB1 = parData.loc[idx,'thetaB'] - parData.loc[idx-1,'thetaB']
        if(diffB1 <= -180.0):
            diffB = diffB1 + 360.0
        elif(diffB1 >= 180.0):
            diffB = diffB1 - 360.0
        else:
            diffB = diffB1
        parData.loc[idx,'wB'] = diffB/(PERIOD)
    parData['wAVG'] = 0.5*(parData['wA']+parData['wB'])

    
    return parData


#### Find Angle Between Swimmers (Account for Quadrant)

In [5]:
def Rotate(xy, theta):
    # https://en.wikipedia.org/wiki/Rotation_matrix#In_two_dimensions
    #First Rotate based on Theta
    #Allocate Arrays
    rotationMatrix = np.zeros((2,2))
    #Calculate rotation matrix
    rotationMatrix[0,0] = np.cos(theta)
    rotationMatrix[0,1] = -1.0*np.sin(theta)
    rotationMatrix[1,0] = np.sin(theta)
    rotationMatrix[1,1] = np.cos(theta) 
    return rotationMatrix.dot(xy)

def Translate(xy, offset):
    return xy + offset

def CalcSystemCM(ACM,BCM):
    xCM_comb = 0.5*(ACM[0]+BCM[0])
    yCM_comb = 0.5*(ACM[1]+BCM[1])
    return (xCM_comb, yCM_comb)

class Swimmer:
    def __init__(self,data):
        self.data  = data
        self.xU, self.yU = data['xU'], data['yU']
        self.xL, self.yL = data['xL'], data['yL']
        self.time = data['time']/2.0
        self.CM = np.zeros((2,len(self.time)))
        self.theta = np.zeros(len(self.time))
        self.normY = np.zeros((2,len(self.time)))
        self.normX = np.zeros((2,len(self.time)))
        self.labH = np.zeros((2,len(self.time)))
        self.Hx = np.zeros(len(self.time))
        self.Hy = np.zeros(len(self.time))
        self.rot_norm = np.zeros((2,len(self.time)))
        self.theta_BW = np.zeros(len(self.time))
        self.CalcCM()
        #print('CM calculated')
        self.CalcLabAngle()
        #print('theta Calculated')
        #print('theta = ',self.theta)
    
    def get_theta(self):
        return self.theta

    def get_labH(self):
        return (self.labHx/RADIUSSMALL, self.labHy/RADIUSSMALL)
    
    def get_norms(self):
        return (self.normX, self.normY)
        
    def CalcCM(self):
        self.xCM = 0.8*self.xU + 0.2*self.xL
        self.yCM = 0.8*self.yU + 0.2*self.yL
        self.CM[0] = self.xCM
        self.CM[1] = self.yCM
        #print('CM = ',self.CM)
        
    def CalcLabAngle(self):
        #Find swimming axis (normal y-axis)
        self.labX   = self.xU - self.xL
        self.labY   = self.yU - self.yL
        self.length = np.hypot(self.labX,self.labY)
        self.normY[0] = self.labX/self.length
        self.normY[1] = self.labY/self.length
        #self.normY  = np.array([self.labX/self.length,self.labY,self.length])
        #2) Calculate Theta
        for idx in range(len(self.time)):
            if(self.normY[1,idx] >= 0.0):
                self.theta[idx] = np.arccos(self.normY[0,idx])
                #Theta_a[idx] = np.arccos(norm_aX[idx])
            else:
                self.theta[idx] = -1.0*np.arccos(self.normY[0,idx])+2.0*np.pi
                #Theta_a[idx] = -1.0*np.arccos(norm_aX[idx])+2.0*np.pi
        #Calculate perpendicular axis (swimmer's x-axis)
        self.normX[0] = np.cos(self.theta - 0.5*np.pi)
        self.normX[1] = np.sin(self.theta - 0.5*np.pi)
        
    def CalcHxHyTheta(self,swimmerA):
        #Here, we will calculate three parameters
        #1) Hx: In the swimmer A's reference frame
        #2) Hy: In the swimmer A's reference frame
        #3) Theta_BW: Angle between swimmer A and swimmerB (w/ respect to swimmer A y-axis)
        #But first, we must calculate in lab frame for some
        #print('yCM = ',self.yCM)
        #print('A_yCM = ',swimmerA.yCM)
        self.labHx = self.xCM - swimmerA.xCM
        self.labHy = self.yCM - swimmerA.yCM
        self.Hmag  = np.hypot(self.labHx,self.labHy)
        self.labH[0], self.labH[1] = self.labHx, self.labHy
        #1) Hx: In swimmer A's reference frame
        #2) Hy: In swimmer A's reference frame
        self.Projection(swimmerA.normX,swimmerA.normY,swimmerA.CM)
        
        #3) Rotate A and B ccw by 2pi - Theta_a
        self.rotateAngle = 2.0*np.pi - swimmerA.theta
        #RotationMatrix (def RotationMatrix)
        #Create Swimmer Position Arrays ??
        #self.normY
        
        for idx in range(len(self.length)):
            self.rot_norm[:,idx] = Rotate(self.normY[:,idx],self.rotateAngle[idx])
            swimmerA.rot_norm[:,idx] = Rotate(swimmerA.normY[:,idx],self.rotateAngle[idx])
            if(self.rot_norm[1,idx] >= 0.0):
                self.theta_BW[idx] = np.arccos(swimmerA.rot_norm[:,idx].dot(self.rot_norm[:,idx]))%(2.0*np.pi)
            else:
                self.theta_BW[idx] = (-1.0*np.arccos(swimmerA.rot_norm[:,idx].dot(self.rot_norm[:,idx]))+2.0*np.pi)%(2.0*np.pi)
        
        return (self.Hx/RADIUSSMALL, self.Hy/RADIUSSMALL, np.hypot(self.Hx,self.Hy)/RADIUSSMALL, self.theta_BW/np.pi)

    def Projection(self,normX,normY,A_CM):
        #Here we find the distance component for swimmer A's axes
        #print('normY = ',normY)
        #print('CM = ',self.CM)
        for idx in range(len(self.time)):
            '''print('normY[:,idx] = ',normY[:,idx])
            print('CM[idx] = ',self.CM[:,idx])
            dot = np.dot(normY[:,idx],self.CM[:,idx])
            print('dot = ',dot)
            ycoord = dot*normY[:,idx]
            print('ycoord = ',ycoord)'''
            self.Hx[idx] = np.dot(normX[:,idx],(self.CM[:,idx]-A_CM[:,idx]))
            self.Hy[idx] = np.dot(normY[:,idx],(self.CM[:,idx]-A_CM[:,idx]))


#### Create dataframe of all pos data for all time for all sims

In [6]:
#The main goal of this script is to create an H-Theta Phase Space of all
#simulations for every period elapsed.
#Example: 20s of sim time = 200 periods. 200 H-Theta Plots
#We may find that we can combine the H-Theta data after steady state has been reached
#1) For each simulation, store position data for every period (nothing in between)
#2) Calculate separation distance between large spheres (H)
#3) Calculate relative heading (Theta)
#4) Calculate deltaH and deltaTheta (change after one Period)
#5) Plot H vs Theta (Polar) for each period

nRe = len(ReList)

count = 0
dict_allData = {'Re':[],'Hx':[],'Hy':[],'Hmag':[],'Theta':[],'time':[]}
df_allData = pd.DataFrame(data=dict_allData)
for Re in ReList:
    cwd_POSDATA = cwd_PYTHON+'/../PosData/O/Re'+Re+'/'
    strPrint = 'Re:'+Re
    print(strPrint)
    #Get All sim data and store
    simData = StoreData(cwd_POSDATA)
    #Add Simulation Parameter identifiers
    simData['Re'] = float(Re)
    dict_sim = {'Hx':simData['Hx'],'Hy':simData['Hy'],'Hmag':simData['Hmag'],'Theta':simData['theta_deg'],
                'xCM_comb':simData['xCM_comb'],'yCM_comb':simData['yCM_comb'],
                'xCMA':simData['xCMA'],'yCMA':simData['yCMA'],
                  'xCMB':simData['xCMB'],'yCMB':simData['yCMB'],
                'wA':simData['wA'],'wB':simData['wB'],'wAVG':simData['wAVG'],
                'Re':simData['Re'],'time':simData['time']}
    df_sim = pd.DataFrame(data=dict_sim)
    df_allData = pd.concat([df_allData,df_sim],ignore_index=True)
    count += 1
    #print('Re ='+Re)
allData_sweep = df_allData.copy()
allData_sweep = allData_sweep.sort_values(by=['Re','time'])
#allData.to_csv(cwd_PYTHON+'/allData_v2.csv',index=False,sep=' ')
print(len(allData_sweep['time']))
print('count = ',count)
print('Storing Data is complete! Onto Analysis')
                

Re:0.5
Re:1.0


/Users/thomas/anaconda3/envs/tf-research/lib/python3.6/site-packages/ipykernel_launcher.py:31: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Re:2.0
Re:3.0
Re:4.0
Re:5.0
Re:5.5
Re:6.0
Re:6.5
Re:7.0
Re:7.5
Re:10.0
Re:12.5
Re:15.0
Re:17.5
Re:20.0
Re:25.0
Re:30.0
Re:35.0
Re:40.0
Re:50.0
Re:60.0
4283
count =  22
Storing Data is complete! Onto Analysis


#### Plot the following quantities:
##### (Theta_bw vs time), (Hmag vs time), (v_comb vs time)
#### After all are found, then get plateaus (long-time behavior) and plot vs Re
#### Get mean and std of each parameter being tracked
##### (Theta_bw vs Re), (Hmag vs. Re), (v_comb vs Re)

In [7]:
#Plotting Functions

def LabelSubPlots(ax, Re):

    csfont = {'fontname':'Times New Roman'}
    ax[0].set_title(r'$\theta_{bw}$ vs. time: Re =  '+Re,fontsize=15,**csfont)
    ax[0].set_xlabel(r'time (s)',fontsize=12,**csfont)
    ax[0].set_ylabel(r'$\theta_{bw}$',fontsize=12,**csfont)
    ax[1].set_title(r'$H_{bw}$ vs. time: Re =  '+Re,fontsize=15,**csfont)
    ax[1].set_xlabel(r'time (s)',fontsize=12,**csfont)
    ax[1].set_ylabel(r'$H_{bw}$',fontsize=12,**csfont)
    ax[2].set_title(r'$y_{CM,comb}$ vs. $x_{CM,comb}$: Re =  '+Re,fontsize=15,**csfont)
    ax[2].set_xlabel(r'$x_{CM,comb}$ (r)',fontsize=12,**csfont)
    ax[2].set_ylabel(r'$y_{CM,comb}$ (r)',fontsize=12,**csfont)
    ax[3].set_title(r'$\omega_{AVG}$ vs. time: Re =  '+Re,fontsize=15,**csfont)
    ax[3].set_xlabel(r'time (s)',fontsize=12,**csfont)
    ax[3].set_ylabel(r'$\omega_{AVG}$ (deg/s)',fontsize=12,**csfont)
    ax[0].axis([0.0,20.0,150.0,210.0])
    ax[1].axis([0.0,20.0,5.0,20.0])
    ax[2].axis([-10.0,10.0,-10.0,10.0])
    ax[3].set_xlim(0.0,20.0)
    
    return ax

def PlotvsTime(ax,data,var1,var2,Re):
    ax.scatter(data[var1],data[var2],c='k',s=9)
    ax.plot(data[var1],data[var2],c='k')
    return ax
    

In [8]:
%matplotlib inline

#Plot 1) Theta_bw vs time for each Re
#     2) Dist_bw vs time for each Re
#     3) vel_combined vs time for each Re (Might need to extract new info for this)
#fig, ax = plt.subplots(nrows=1,ncols=3,num=1,figsize=(19,6),dpi=250)
for Re in ReList:
    fig, ax = plt.subplots(nrows=1,ncols=4,num=1,figsize=(25,6),dpi=250)
    ReData = allData_sweep[allData_sweep['Re'] == float(Re)].copy()
    ReData['Theta'] = 360.0 - ReData['Theta']
    #print(ReData.head())
    ax = LabelSubPlots(ax,Re)
    ax[0] = PlotvsTime(ax[0],ReData,'time','Theta',Re)
    ax[1] = PlotvsTime(ax[1],ReData,'time','Hmag',Re)
    ax[2] = PlotvsTime(ax[2],ReData,'xCM_comb','yCM_comb',Re)
    #ax[2] = PlotvsTime(ax[2],ReData,'xCMA','yCMA',Re)
    #ax[2] = PlotvsTime(ax[2],ReData,'xCMB','yCMB',Re)
    ax[3] = PlotvsTime(ax[3],ReData,'time','wAVG',Re)
    fig.tight_layout()
    #plt.show()
    #sys.exit(0)
    pathlib.Path(cwd_PYTHON+'/../Figures/O/vsTime/').mkdir(parents=True, exist_ok=True)
    fig.savefig(cwd_PYTHON+'/../Figures/O/vsTime/MonitoredParameters_Re'+Re+'.png')
    fig.clf()
    print('Re = %s done'%Re)
plt.close()


Re = 0.5 done
Re = 1.0 done
Re = 2.0 done
Re = 3.0 done
Re = 4.0 done
Re = 5.0 done
Re = 5.5 done
Re = 6.0 done
Re = 6.5 done
Re = 7.0 done
Re = 7.5 done
Re = 10.0 done
Re = 12.5 done
Re = 15.0 done
Re = 17.5 done
Re = 20.0 done
Re = 25.0 done
Re = 30.0 done
Re = 35.0 done
Re = 40.0 done
Re = 50.0 done
Re = 60.0 done


In [9]:
from matplotlib.patches import Circle
def AddDiscsToPlot(ax):
    RADIUS_LARGE = 2.0
    RADIUS_SMALL = 1.0
    #Add Discs
    circle1 = Circle((-5.0, 0.0), RADIUS_LARGE, facecolor=(0.0,)*3,
                     linewidth=1,alpha=1.0)#,zorder=6)
    ax.add_patch(circle1)
    circle2 = Circle((-5.0, -5.0), RADIUS_SMALL, facecolor=(0.0,)*3,
                     linewidth=1,alpha=1.0)#,zorder=6)
    ax.add_patch(circle2)
    circle3 = Circle((5.0, -5.0), RADIUS_LARGE, facecolor=(0.0,)*3,
                             linewidth=1,alpha=1.0)#,zorder=6)
    ax.add_patch(circle3)
    circle4 = Circle((5.0, 0.0), RADIUS_SMALL, facecolor=(0.0,)*3,
                     linewidth=1,alpha=1.0)#,zorder=6)
    ax.add_patch(circle4)
    return

#Trajectories
#Plot both swimmer trajectories
csfont = {'fontname':'Times New Roman'}
for Re in ReList:
    fig, ax = plt.subplots(nrows=1,ncols=1,num=1,figsize=(6,6),dpi=250)
    ReData = allData_sweep[allData_sweep['Re'] == float(Re)].copy()
    ReData = ReData.reset_index(drop=True)
    #ax.set_title(r'Trajectories: Swimmer A () vs. $H_{bw}$: Re = '+Re,fontsize=15,**csfont)
    ax.set_xlabel(r'x (r)',fontsize=12,**csfont)
    ax.set_ylabel(r'y (r)',fontsize=12,**csfont)
    ax.scatter(ReData.loc[0,'xCMA'],ReData.loc[0,'yCMA'],c='r',s=25,zorder=5)
    ax.scatter(ReData.loc[0,'xCMB'],ReData.loc[0,'yCMB'],c='r',s=25,zorder=5)
    ax.scatter(ReData['xCMA'],ReData['yCMA'],c=ReData['time'],cmap='viridis',s=5,zorder=3)
    ax.scatter(ReData['xCMB'],ReData['yCMB'],c=ReData['time'],cmap='viridis',s=5,zorder=3)
    AddDiscsToPlot(ax)
    if(float(Re) < 6.0):
        ax.axis([-20.0,20.0,-20.0,20.0])
    else:
        ax.axis([-10.0,10.0,-10.0,10.0])
    fig.tight_layout()
    #plt.show()
    #sys.exit(0)
    pathlib.Path(cwd_PYTHON+'/../Figures/O/Trajectories/').mkdir(parents=True, exist_ok=True)
    fig.savefig(cwd_PYTHON+'/../Figures/O/Trajectories/Trajectories_Re'+Re+'.png')
    fig.clf()
    print('Re = %s done'%Re)
plt.close()


Re = 0.5 done
Re = 1.0 done
Re = 2.0 done
Re = 3.0 done
Re = 4.0 done
Re = 5.0 done
Re = 5.5 done
Re = 6.0 done
Re = 6.5 done
Re = 7.0 done
Re = 7.5 done
Re = 10.0 done
Re = 12.5 done
Re = 15.0 done
Re = 17.5 done
Re = 20.0 done
Re = 25.0 done
Re = 30.0 done
Re = 35.0 done
Re = 40.0 done
Re = 50.0 done
Re = 60.0 done


In [ ]:
#Limit Cycle Stuff
#Plot theta_bw vs l_bw for each Re
csfont = {'fontname':'Times New Roman'}
for Re in ReList:
    fig, ax = plt.subplots(nrows=1,ncols=1,num=1,figsize=(6,6),dpi=250)
    ReData = allData_sweep[allData_sweep['Re'] == float(Re)].copy()
    ax.set_title(r'Limit Cycle: $\omega_{AVG}$ vs. $H_{bw}$: Re = '+Re,fontsize=15,**csfont)
    ax.set_xlabel(r'$H_{bw}$ (r)',fontsize=12,**csfont)
    ax.set_ylabel(r'$\omega_{AVG}$',fontsize=12,**csfont)
    ax.scatter(ReData['Hmag'],ReData['wAVG'],c=ReData['time'],cmap='viridis',s=5)
    #ax.axis([5.0,7.0,175.0,185.0])
    ax.set_xlim(5.0,7.0)
    fig.tight_layout()
    #plt.show()
    #sys.exit(0)
    pathlib.Path(cwd_PYTHON+'/../Figures/O/LimitCycle/').mkdir(parents=True, exist_ok=True)
    fig.savefig(cwd_PYTHON+'/../Figures/O/LimitCycle/LimitCycle_Re'+Re+'.png')
    fig.clf()
    print('Re = %s done'%Re)
plt.close()

In [11]:
from scipy import stats

#Construct time_ES list
ReList_orbit = [6.0,6.5,7.0,7.5,10.0,12.5,15.0,17.5,20.0,25.0,30.0]
time_ES = [12.0,15.0,6.0,5.0,4.0,4.0,3.5,3.5,3.0,3.0,3.0]
#Given the time_ES (time the end state begins), calculate the mean and std dev of the following parameters
#theta_BW, H_BW, v_comb for each Re
#Save them in np arrays
#Plot parameters vs Re

#Arrays (0 = mean, 1 = std dev)
thetaBW_stats = np.zeros((11,2))
HBW_stats     = np.zeros((11,2))
omega_stats   = np.zeros((11,2))

for idx in range(len(time_ES)):
    ReData = allData_sweep[allData_sweep['Re'] == float(ReList_orbit[idx])].copy()
    ReData = ReData[ReData['time'] >= time_ES[idx]].copy()
    ReData = ReData.reset_index(drop=True)
    #Remove any outliers (H_BW jump)
    #ReData['zscore'] = np.abs(stats.zscore(ReData['Hmag']))
    #ReData = ReData[ReData['zscore'] < 3]
    #ReData = ReData.reset_index(drop=True)
    ReData['Theta'] = 360.0 - ReData['Theta']
    #Calculate Delta for each Re
    delta = np.sqrt(RADIUSSMALL*RADIUSLARGE*0.8*0.8/float(ReList[idx]))
    #Means and stds
    thetaBW_stats[idx,0] = ReData['Theta'].mean()
    thetaBW_stats[idx,1] = ReData['Theta'].std()
    HBW_stats[idx,0]     = ReData['Hmag'].mean()*(delta/RADIUSSMALL)
    HBW_stats[idx,1]     = ReData['Hmag'].std()*(delta/RADIUSSMALL)
    omega_stats[idx,0]   = ReData['wAVG'].mean()
    omega_stats[idx,1]   = ReData['wAVG'].std()

#Plot parameters vs. Re (include std as error bar)
csfont = {'fontname':'Times New Roman'}
fig, ax = plt.subplots(nrows=1,ncols=3,num=2,figsize=(19,6),dpi=250)
ax[0].set_title(r'$\theta_{bw}$ vs. Re: 2$\sigma$',fontsize=15,**csfont)
ax[0].set_xlabel(r'Re',fontsize=12,**csfont)
ax[0].set_ylabel(r'$\theta_{bw}$',fontsize=12,**csfont)
ax[1].set_title(r'$H_{bw}$ vs. Re: 2$\sigma$',fontsize=15,**csfont)
ax[1].set_xlabel(r'Re',fontsize=12,**csfont)
ax[1].set_ylabel(r'$H_{bw}$ ($\delta$)',fontsize=12,**csfont)
ax[2].set_title(r'$\omega_{AVG}$ vs. Re: 2$\sigma$',fontsize=15,**csfont)
ax[2].set_xlabel(r'Re',fontsize=12,**csfont)
ax[2].set_ylabel(r'$\omega_{AVG}$ (deg/s)',fontsize=12,**csfont)
#Plot Parameters with error bars
ax[0].errorbar(ReList_orbit, thetaBW_stats[:,0], yerr=2.0*thetaBW_stats[:,1],c='k')
ax[0].scatter(ReList_orbit, thetaBW_stats[:,0],c='k',s=12)
ax[1].errorbar(ReList_orbit, HBW_stats[:,0], yerr=2.0*HBW_stats[:,1],c='k')
ax[1].scatter(ReList_orbit, HBW_stats[:,0],c='k',s=12)
ax[2].errorbar(ReList_orbit, omega_stats[:,0], yerr=2.0*omega_stats[:,1],c='k')
ax[2].scatter(ReList_orbit, omega_stats[:,0],c='k',s=12)
ax[0].axis([5.0,35.0,170.0,190.0])
ax[1].axis([5.0,35.0,0.5,20.0])
#ax[2].axis([0.0,60.0,-0.5,0.5])
ax[2].set_xlim(5.0,35.0)
fig.tight_layout()
pathlib.Path(cwd_PYTHON+'/../Figures/O/').mkdir(parents=True, exist_ok=True)
fig.savefig(cwd_PYTHON+'/../Figures/O/Params_vs_Re_delta.png')
fig.clf()
plt.close()
print('Plotting vs. Re is complete!')

#Plot Log-Log H_BW vs. Re

deltaValues = [np.sqrt(RADIUSSMALL*RADIUSLARGE*0.8*0.8/float(a)) for a in ReList_orbit]

#Find Relationship between H_bw and \delta
#Use HBW_stats and deltaValues
deltaArray = np.array(deltaValues)
nRe = len(deltaArray)
linearDelta = deltaArray[3:nRe].copy()
print(linearDelta)
linearHBW = HBW_stats[3:nRe,0]
#Relationship for first part
linDelta1 = deltaArray[0:3].copy()
linHBW1   = HBW_stats[0:3,0].copy()


#Find Linear Regression
slope, intercept, r_value, p_value, std_err = stats.linregress(np.log10(linearDelta),np.log10(linearHBW))
print("="*40)
print(r'Power Law: $\delta$ exponent')
print('N = ',len(linearDelta))
print('exponent = %.3f'%slope)
print('intercept = %.3f'%intercept)
print('r_value = %.3f'%r_value)

#Find linear regression: First part
slope1, intercept1, r_value1, p_value1, std_err1 = stats.linregress(np.log10(linDelta1),np.log10(linHBW1))
print("="*40)
print(r'Power Law: $\delta$ exponent')
print('N = ',len(linDelta1))
print('exponent = %.3f'%slope1)
print('intercept = %.3f'%intercept1)
print('r_value = %.3f'%r_value1)

fig, ax = plt.subplots(nrows=1,ncols=1,num=2,figsize=(6,6),dpi=250)
ax.set_title(r'$H_{bw}$ vs. Re: log-log: $H_{bw} \propto$ $\delta^{%.2f}$'%slope,fontsize=15,**csfont)
ax.set_xlabel(r'Re',fontsize=12,**csfont)
ax.set_ylabel(r'$H_{bw}$ ($\delta$)',fontsize=12,**csfont)
ax.loglog(ReList_orbit,HBW_stats[:,0],c='k')
ax.scatter(ReList_orbit,HBW_stats[:,0],c='k')
#Plot Fit 2
yFit = 10.0**(intercept)*deltaArray**slope
ax.loglog(ReList_orbit,yFit,c='r',ls='--')
#Plot Fit 1
yFit1 = 10.0**(intercept1)*deltaArray**slope1
ax.loglog(ReList_orbit,yFit1,c='b',ls='--')
ax.axis([5.0,35.0,0.5,20.0])
fig.tight_layout()
pathlib.Path(cwd_PYTHON+'/../Figures/O/').mkdir(parents=True, exist_ok=True)
fig.savefig(cwd_PYTHON+'/../Figures/O/H_vs_Re_LL_delta.png')
fig.clf()
plt.close()
print('Plotting log-log is complete')


Plotting vs. Re is complete!
[0.00041312 0.00035777 0.00032    0.00029212 0.00027045 0.00025298
 0.00022627 0.00020656]
Power Law: $\delta$ exponent
N =  8
exponent = 0.862
intercept = 3.514
r_value = 0.979
Power Law: $\delta$ exponent
N =  3
exponent = 9.763
intercept = 33.611
r_value = 1.000
Plotting log-log is complete
